# 2 - Run Optimising Sub-pattern Recognition: Memory Capacity (Generates Data for Figures 9 and 10)
 

This notebook re-generates data for the section *Section 9.1: Optimising Sub-pattern Recognition* and caches it.

**To just plot the results:** 
If you would like to just plot the pre-generated paper results, there is no need to re-generate the results as they are cached in `examples/experimental_results/experiment_A_sub_pattern`. In this case, simply use the Jupyter notebook `3 - Experiment_A_sub_pattern - Plot Figs 9 and 10.ipynb`. 

**Where the data will be saved:** The results from this test willl be saved in the existing Excel files in the `examples/experimental_results/experiment_A_sub_pattern` folder and overwrite the sheet named `latest_data`. The data generated for the paper is on the `paper_data` sheet, which will not be overwritten. You can select the data to plot (the latest or the paper data) by selecting the correct sheet name in `3 - Experiment_A_sub_pattern - Plot Figs 9 and 10.ipynb`. 

This notebook explores the effect on the sub-pattern recognition step through the two optimisation techniques: 

* Optimising the hidden neuron threshold
* Optimising the feature to hidden neuron connections

To examine the effect of the sub-pattern recognition optimisations on ESAM network memory capacity, the following experiment is used:

<table>
<tr><td>

| Problem A    |          |
| :----------- | :------  | 
| $f$          | 900      | 
| $m$          | 0 - 25,000 | 
| $s_m$        | 0.22     | 
| $s_n$        | 0        | 
</td><td>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
</td><td>

| Network A (Baseline) |          |
| :--------------------| :------  | 
| $h$                  | 2        | 
| $\theta$             | 12       | 
| $s^+_{f\_h}$         | 0.1      | 
| $\sigma_{h\_pre+}$   | `FixedProbability` | 
| $s^+_{h\_f}$         | 1     | 
| $s^-_{h\_f}$         | 1     | 
| $\sigma_{f\_pre+}$, $\sigma_{f\_pre-}$  |  `FixedProbability`       | 
| $e$                  | 3        | 

</td></tr> </table>

Note that this is a very simple verification of `f(x)=x` as there is no noise present in the recall signals. It is based on the baseline experiment presented in [1] (figure 4a).

The accuracy of four different networks are compared in the graphs as the number of memories $m$ varies:

* Network A - the baseline network from [1] (as described above).
* Network A with $\theta$ optimised for each test as described in section 6.2.
* Network A with sub-pattern recognition (feature to hidden) connections optimised as described in section 6.3.
* Network A with $\theta$ and the connections optimised

The accuracy of each network is tested with increasing numbers of memories. This is the only variable that changes during the tests.



[1] Hoffmann, H. (2019). Sparse Associative Memory. *Neural Computation*

## Dependencies and problem/network definition

Import the dependencies and define the tests to run. 

### Dependencies

In [1]:
import os
import sys
sys.path.append('../src')


In [2]:
output_base_dir = '.'+os.sep+'experiment_results'
test_name = 'experiment_A_sub_pattern'

### Define the problem space and base network

Provide the default values for the problem space and the network. The tests will be variations 
on these defaults.

In [3]:
f = 900
s_m = 200/f
problem_space_default = {'f': f,
                         'm': 200,
                         's_m': s_m,
                         's_n': 0}

network_default = {'f': f,
                   'h': 2,
                   'f_h_sparsity': 0.1,
                   'h_f_sparsity_e': 1,
                   'h_f_sparsity_i': 1,
                   'e': 3,
                   'h_thresh': 12,
                   'f_h_conn_type': 'FixedProbability',
                   'h_f_conn_type': 'FixedProbability',
                   'debug': False
                   }


### Define the variable that will change (the x-axis on the final plots)

In this case, the variable that is changed is the number of memories `m`.

In [4]:
# Below is the full test, as in the paper.
vary_memories = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 5000, 10000, 15000, 20000]

# The full test will take a long time to run. You can experiment with fewer memories.
# You can also reduce the number of simulations that are run per memory variation - see the 
# 'Run the Tests' section below. 
vary_memories = [100, 200, 300]

### Define the networks to be tested. 

Each network will be tested multiple times for each of the variable options.

The `problem_vary` and `network_static` dictionary keys indicate that the variation betweeen the tests (for each network) is the problem space - specifically in this case, the number of memories `m`

There are 4 network varieties:
* `original`: the original network from the paper.
* `opt_thresh`: the original network with the hidden neuron threshold optimised for each of the varying memory scenarios.
* `opt_conns`: the original network with the feature to hidden (sub-pattern) connections optimised.
* `opt_thresh_conns`: the original network combining the optimisations of both `opt_thresh` and `opt_conns`.

In [5]:
import copy

original={
            'problem_vary': problem_space_default.copy(),
            'network_static': network_default.copy(),
            'plot_params':
                {'variable_column': 'm',
                 'variable_column_name': 'Number of memories',
                 'variable_is_integer': False}
         }


# Override with the variable parameter. Specifying an array for this parameter will
# indicate to the test runner that there are multiple scenarios to be tested.
original['problem_vary'][original['plot_params']['variable_column']] = vary_memories

opt_thresh = copy.deepcopy(original)
opt_thresh['network_static']['h_thresh'] = -1     # -1 means optimise the threshold

opt_conns = copy.deepcopy(original)
opt_conns['network_static']['f_h_conn_type'] = 'FixedNumberPre'

opt_thresh_conns = copy.deepcopy(opt_thresh)
opt_thresh_conns['network_static']['f_h_conn_type'] = 'FixedNumberPre'


Put all the network descriptions into a single dictionary for the test runner.

In [6]:
all_tests = {'original': original,
             'opt_thresh': opt_thresh,
             'opt_conns': opt_conns,
             'opt_thresh_conns': opt_thresh_conns
            }

## Run the tests

`num_tests` is the number of networks for each type of test that will be created. Despite having the same network definition, each will vary due to randomness in the data and network connections.

`simulations_per_test` is the number of recalls that will be run on each test network.

The total number of tests that will be run is: `number of test networks (4) * len(vary_m) * num_tests * simulations_per_test`

This test is classified as `vary_problem` because the network remains static for each optimisation test, but the problem is varied. 

In [7]:
num_tests = 10
sims_per_test = 10
output_dir = output_base_dir + os.sep + test_name

In [8]:
from simulation_scripts.test_suite_runner import TestSuiteRunner

tsr = TestSuiteRunner(network_default=network_default,
                      problem_space_default=problem_space_default,
                      output_dir=output_dir)


tsr.run_vary_problem_space_test_suite(all_tests = all_tests,
                                      num_tests = num_tests,
                                      sims_per_test = sims_per_test)


Running test set: original defined as: 
    Static network params:    {'f': 900, 'h': 2, 'f_h_sparsity': 0.1, 'h_f_sparsity_e': 1, 'h_f_sparsity_i': 1, 'e': 3, 'h_thresh': 12, 'f_h_conn_type': 'FixedProbability', 'h_f_conn_type': 'FixedProbability', 'debug': False}
    Varying problem space:    {'f': 900, 'm': [100, 200, 300], 's_m': 0.2222222222222222, 's_n': 0}

Number of tests per scenario: 10 
Number of simulations per test: 10 

--------------------------------------------------------------------------------------------
Simulation 1 of 10 for test 1 of 3

======= Generating Simulation Data =======
      Number of features (feature neurons): 900
      Number of memories:                   100
      Number of simulations:                10
      Signal length:                        200
      Bits to flip for noise:               0
  Learning phase...

=========== ESAM Recall Network =============
    Num features:                   900
    F to H connection sparsity:     0.1
     

In [9]:
print('Experiment complete, data stored in:\n')
print('  {:80s}    {:10s}\n'.format('File', 'Sheet'))

for test_file, _ in all_tests.items():
    excel_file = output_base_dir + os.sep + test_name + os.sep + test_file + '.xlsx'
    print('  {:80s}    {:10s}'.format(excel_file, 'latest'))

Experiment complete, data stored in:

  File                                                                                Sheet     

  ./experiment_results/experiment_A_sub_pattern/original.xlsx                         latest    
  ./experiment_results/experiment_A_sub_pattern/opt_thresh.xlsx                       latest    
  ./experiment_results/experiment_A_sub_pattern/opt_conns.xlsx                        latest    
  ./experiment_results/experiment_A_sub_pattern/opt_thresh_conns.xlsx                 latest    


## Next steps: Plotting the results

Use the Jupyter notebook `3 - Experiment_A_sub_pattern - Plot Figs 9 and 10.ipynb` to plot the results.

You will need to update the `data_sheet` variable to `latest_data` as described in the notebook, else the pre-generated paper data will be plotted.